In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time
import random
import psycopg2

In [ ]:
def scrapping():
    driver = webdriver.Chrome()
    driver.get(url="https://www.pccomponentes.com/componentes")
    driver.maximize_window() 
    
    
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[@class='btn btn-block btn-primary  btn-lg m-t-1 accept-cookie']")))
    while True:
        try:
            #Accept Cookies
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-block btn-primary  btn-lg m-t-1 accept-cookie']"))).click() 
            print("Cookies aceptadas")
            break
        except TimeoutException:
            break
            
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@href='/placas-base']"))).click()
    
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-secondary btn-block btn-lg btn-more']")))
    while True:
        try:
            #Accept Cookies
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-secondary btn-block btn-lg btn-more']"))).click() 
            print("Ver más...")
            break
        except TimeoutException:
            break
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    #Randoms mouse movements
    action = ActionChains(driver)
    
    
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
        # Wait to load page
        time.sleep(0.5)
        
    
        action.move_by_offset(random.randint(0,10),random.randint(0,10));
        action.perform();
        time.sleep(0.1)
        
        #Scroll top
        driver.execute_script("window.scrollTo(0, 0);")
                              
        # Wait to load page
        time.sleep(0.5)
    
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height
        
    div = driver.find_elements_by_xpath("//article[@data-category='Placas Base']")
    
    global articles_list, categories_list, prices_list, stock_list
    articles_list = []
    categories_list = []
    prices_list = []
    stock_list = []
    
    
    for i in div:
        articles_list.append(i.get_attribute('data-name'))
        categories_list.append(i.get_attribute('data-category'))
        prices_list.append(i.get_attribute('data-price'))
        stock_list.append(i.get_attribute('data-stock-web'))
        
    return articles_list, categories_list, prices_list, stock_list

scrapping()

In [30]:
def convert_dataframe(articles_list, categories_list, prices_list, stock_list):
    
    global data
    
    data_tuples = list(zip(articles_list, categories_list, prices_list, stock_list))
    data = pd.DataFrame(data_tuples, columns=['Articulo','Categoria', 'Precio', 'Stock'])
    
    
    return data

In [32]:
convert_dataframe(articles_list, categories_list, prices_list, stock_list)

,Articulo,Categoria,Precio,Stock
0,Gigabyte B450M DS3H,Placas Base,72.99,1
1,MSI B450 Gaming Plus MAX,Placas Base,145.4,1
2,MSI Z390-A PRO,Placas Base,129.91,1
3,MSI B450M-A PRO MAX,Placas Base,60.98,1
4,MSI X470 Gaming Plus Max,Placas Base,131.9,1
...,...,...,...,...
435,Placa Base MSI X470 Gaming Pro Reacondicionado,Placas Base,96.8,4
436,AsRock Z390 Taichi,Placas Base,279.93,1
437,Gigabyte X299X Designare 10G,Placas Base,695.25,1
438,Asrock Fatal1ty X370 Gaming X,Placas Base,137.5,1


AttributeError: 'DataFrame' object has no attribute 'save'

In [ ]:
from configparser import ConfigParser


def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [ ]:
config(filename='database.ini', section='postgresql')